# Decision Tree Regression From Scratch

Decision Tree Regression uses a tree structure to predict continuous values by recursively splitting the feature space.

## Key Concepts:
- **MSE Splitting Criterion**: Minimize mean squared error
- **Variance Reduction**: Split to reduce variance in child nodes
- **Leaf Predictions**: Mean of target values in leaf
- **Non-parametric**: No assumptions about data distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

## 1. Mathematical Foundation

### Mean Squared Error (MSE):
$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \bar{y})^2$$

### Variance Reduction:
$$\Delta = MSE_{parent} - \left(\frac{n_{left}}{n} MSE_{left} + \frac{n_{right}}{n} MSE_{right}\right)$$

### Leaf Prediction:
$$\hat{y} = \frac{1}{n} \sum_{i=1}^{n} y_i$$

(Mean of all target values in the leaf)

## 2. Implementation

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature      # Feature index to split on
        self.threshold = threshold  # Threshold value for split
        self.value = value         # Prediction value (for leaf nodes)
        self.left = left           # Left child
        self.right = right         # Right child

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        """
        Initialize Decision Tree Regressor
        
        Parameters:
        -----------
        max_depth : int or None
            Maximum depth of the tree
        min_samples_split : int
            Minimum samples required to split a node
        min_samples_leaf : int
            Minimum samples required in a leaf node
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None
    
    def fit(self, X, y):
        """
        Fit the decision tree
        """
        self.root = self._build_tree(X, y, depth=0)
        return self
    
    def _mse(self, y):
        """
        Calculate mean squared error
        """
        if len(y) == 0:
            return 0
        return np.var(y) * len(y)  # Variance * n = sum of squared deviations
    
    def _variance_reduction(self, y, y_left, y_right):
        """
        Calculate variance reduction from a split
        """
        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        
        if n_left == 0 or n_right == 0:
            return 0
        
        parent_mse = self._mse(y)
        child_mse = (n_left / n) * self._mse(y_left) + (n_right / n) * self._mse(y_right)
        
        return parent_mse - child_mse
    
    def _best_split(self, X, y):
        """
        Find the best split for the data
        """
        best_gain = -1
        best_feature = None
        best_threshold = None
        
        n_features = X.shape[1]
        
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            
            for threshold in thresholds:
                # Split data
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                
                # Check minimum samples constraint
                if np.sum(left_mask) < self.min_samples_leaf or np.sum(right_mask) < self.min_samples_leaf:
                    continue
                
                y_left, y_right = y[left_mask], y[right_mask]
                
                # Calculate variance reduction
                gain = self._variance_reduction(y, y_left, y_right)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_gain
    
    def _build_tree(self, X, y, depth):
        """
        Recursively build the decision tree
        """
        n_samples = len(y)
        
        # Stopping criteria
        if (self.max_depth is not None and depth >= self.max_depth) or \
           n_samples < self.min_samples_split:
            return Node(value=np.mean(y))
        
        # Find best split
        best_feature, best_threshold, best_gain = self._best_split(X, y)
        
        # If no good split found, create leaf
        if best_gain == 0 or best_feature is None:
            return Node(value=np.mean(y))
        
        # Split data
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask
        
        # Recursively build left and right subtrees
        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        
        return Node(feature=best_feature, threshold=best_threshold, 
                   left=left_child, right=right_child)
    
    def _predict_single(self, x, node):
        """
        Predict value for a single sample
        """
        # If leaf node, return value
        if node.value is not None:
            return node.value
        
        # Traverse tree
        if x[node.feature] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)
    
    def predict(self, X):
        """
        Predict values for multiple samples
        """
        return np.array([self._predict_single(x, self.root) for x in X])
    
    def score(self, X, y):
        """
        Calculate R² score
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - (ss_res / ss_tot)

## 3. Testing on Synthetic Data

In [ ]:
# Generate synthetic data
np.random.seed(42)
X, y = make_regression(n_samples=200, n_features=1, noise=20, random_state=42)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")

In [ ]:
# Train Decision Tree Regressor
dt = DecisionTreeRegressor(max_depth=5)
dt.fit(X_train, y_train)

# Make predictions
y_pred_train = dt.predict(X_train)
y_pred_test = dt.predict(X_test)

# Calculate scores
train_score = dt.score(X_train, y_train)
test_score = dt.score(X_test, y_test)

print(f"\nDecision Tree Regressor (max_depth={dt.max_depth})")
print(f"Train R² Score: {train_score:.4f}")
print(f"Test R² Score: {test_score:.4f}")

## 4. Comparison with Scikit-learn

In [ ]:
from sklearn.tree import DecisionTreeRegressor as SklearnDTR

# Train sklearn Decision Tree
sklearn_dt = SklearnDTR(max_depth=5, random_state=42)
sklearn_dt.fit(X_train, y_train)

# Compare scores
sklearn_train_score = sklearn_dt.score(X_train, y_train)
sklearn_test_score = sklearn_dt.score(X_test, y_test)

print("\nComparison:")
print(f"{'Method':<25} {'Train R²':<12} {'Test R²':<12}")
print("-" * 49)
print(f"{'Our Decision Tree':<25} {train_score:<12.4f} {test_score:<12.4f}")
print(f"{'Sklearn Decision Tree':<25} {sklearn_train_score:<12.4f} {sklearn_test_score:<12.4f}")

## 5. Effect of Max Depth

In [ ]:
# Test different max_depth values
depths = range(1, 16)
train_scores = []
test_scores = []

for depth in depths:
    dt = DecisionTreeRegressor(max_depth=depth)
    dt.fit(X_train, y_train)
    
    train_scores.append(dt.score(X_train, y_train))
    test_scores.append(dt.score(X_test, y_test))

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(depths, train_scores, 'o-', label='Train R²', linewidth=2)
plt.plot(depths, test_scores, 's-', label='Test R²', linewidth=2)
plt.xlabel('Max Depth', fontsize=12)
plt.ylabel('R² Score', fontsize=12)
plt.title('Decision Tree Regression: Effect of Max Depth', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

# Find best depth
best_depth = depths[np.argmax(test_scores)]
best_score = max(test_scores)
print(f"\nBest Max Depth: {best_depth}")
print(f"Best Test R²: {best_score:.4f}")

## 6. Visualization of Predictions

In [ ]:
# Train trees with different depths
dt_shallow = DecisionTreeRegressor(max_depth=2)
dt_medium = DecisionTreeRegressor(max_depth=5)
dt_deep = DecisionTreeRegressor(max_depth=10)

dt_shallow.fit(X_train, y_train)
dt_medium.fit(X_train, y_train)
dt_deep.fit(X_train, y_train)

# Create smooth line for predictions
X_line = np.linspace(X_train.min(), X_train.max(), 300).reshape(-1, 1)
y_pred_shallow = dt_shallow.predict(X_line)
y_pred_medium = dt_medium.predict(X_line)
y_pred_deep = dt_deep.predict(X_line)

# Plot
plt.figure(figsize=(15, 5))

for i, (depth, y_pred, title) in enumerate([
    (2, y_pred_shallow, 'Max Depth=2 (Underfitting)'),
    (5, y_pred_medium, 'Max Depth=5 (Good Fit)'),
    (10, y_pred_deep, 'Max Depth=10 (Overfitting)')
]):
    plt.subplot(1, 3, i+1)
    plt.scatter(X_train, y_train, alpha=0.5, s=30, label='Training Data')
    plt.plot(X_line, y_pred, 'r-', linewidth=2, label=f'Decision Tree')
    plt.xlabel('Feature', fontsize=11)
    plt.ylabel('Target', fontsize=11)
    plt.title(title, fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Takeaways

### Advantages:
- ✅ Handles non-linear relationships
- ✅ No feature scaling required
- ✅ Interpretable (can visualize tree)
- ✅ Handles missing values (with modifications)
- ✅ Feature importance available
- ✅ No assumptions about data distribution

### Disadvantages:
- ❌ Prone to overfitting (especially deep trees)
- ❌ High variance (small data changes → different tree)
- ❌ Biased toward features with more levels
- ❌ Not smooth predictions (piecewise constant)
- ❌ Poor extrapolation

### When to Use:
- Non-linear relationships
- Need interpretability
- Mixed feature types
- As base learner for ensemble methods
- Baseline model

### Hyperparameter Tuning:
- **max_depth**: Controls overfitting (lower = less overfitting)
- **min_samples_split**: Minimum samples to split (higher = less overfitting)
- **min_samples_leaf**: Minimum samples in leaf (higher = smoother)

### Comparison with Linear Regression:
- **Decision Tree**: Better for non-linear data, prone to overfitting
- **Linear Regression**: Better for linear data, more stable predictions